This notebook contains an analysis workflow for refining textures in diffraction spectra using MAUD (http://maud.radiographema.eu/)

## Import packages

In [ ]:
import pathlib

from tqdm.notebook import tqdm
import numpy as np

import yaml
from typing import Tuple
from typing import List

## Preparing the spectra

**Notes on caking in DAWN**

Notes on using DAWN to cake the diffraction pattern images are available on the [LightForm Wiki](https://lightform-group.github.io/wiki/tutorials/sxrd-caking).

In DAWN the data must be caked with the x-axis chosen as pixel number (pixels) and saved as a .dat file. It is not possible to adjust data using x-axis as the two-theta angle (degrees).

For caking the data for texture recalculation in MAUD the following parameters are used;

- Import tiffs using either .nxs file (for standard 2 Hz acquisition) or as .dat file (for faster acquisitions) into ‘data slice viewer’ within the ‘processing’ tab.

- Select pilatus, image 2D, finish.

- For **cake remapping**, in ‘processing’ window;
    - Import detector calibration (calibration_output.nxs
    - Threshold mask, set lower to 0.0
    - Cake remapping, azimuthal range 2.5, -357.5, number of azimuthal bins 72, number of bins blank, pixel splitting tick, radial range blank, X axis pixel.
    - Export to text file, file extension .dat, choose output directory, pad with zeros 5

**Notes on caking in pyFAI**

Data can also be caked using [pyFAI](https://pyfai.readthedocs.io/en/latest/#) as explained on the [LightForm Wiki](https://lightform-group.github.io/wiki/tutorials/sxrd-caking-dioptas) using  [pyFAI-integration-caking](https://github.com/LightForm-group/pyFAI-integration-caking) notebook available on GitHub.

## Creating a template ".par" file

Before beginning the batch mode analysis a **template .par file** must be set up in MAUD.
This provides MAUD with information about the beamline setup and also the data to be analysed.

#### Creating and loading beamline calibration

First the beamline calibration data must be loaded into MAUD
- Calibration data for the beamline must be created using a known calibrant. An example of this is shown in the "data/calibration" folder for experiments conducted at Diamond in 2017 and 2021. For example, the "calibration_1200mm.dat" file from 2017 is the specrum of a "CeO2" calibrant.
- To be read into MAUD, the calibration data must be converted to a ".esg" file which can be done [using the script below](#cell1)
- Load the .esg calibration file into MAUD.


#### Creating an instrument type
An **'instrument type'** must then be created in MAUD:
- Follow online videos to define the instrument (https://www.youtube.com/user/MaudRietveldProgram/videos)

*Note, the .dat files have already undergone initial calibration in DAWN or pyFAI, so ImageJ isn't needed to load the tiffs here, which would lead to a very different calibration in MAUD.*

#### Load experimental data into MAUD

- Create a .esg file containing experimental data from the .dat files [using the script below](#cell2).
- Load the data .esg file in MAUD.
- Select the new instrument type defined in the step above.

#### Creating the template .par file

Then, create a **template .par file** in MAUD;

- Set up the refinement;
    - Set data range ~ 1.7 to 9.9 for Diamond 2017 and ~ 2 - 9.3 for Diamond 2021 experiments
    - Add extra background parameter.
    - Add alpha and beta cifs.
    - Set Biso to 0.5
    - Set approximate volume fraction.
    - Manually adjust crystal parameters to fit peak positions.
    - Auto refine background, free scale factors and cell parameters. *Note, leave beam centre fixed.*
    - Auto refine volume fraction.
    - Auto refine crystal size and micro-strain.
    - Auto refine the Caglioti and Gaussian parameters, to better fit the peak shape. *Note, only include this step for high temperature data when the peak shapes are poorly fit*
    - Add E-WIMV texture for alpha-phase. Set ODF resolution to 5, 10 or 15 degree resolution and untick "normalise" on the E-WIMV options. *Note, 15 degree resolution seems to better replicate the actual texture strength at room temperature.*
    - Fix crystal size and micro-strain and turn on texture refinement (but, don't click refine).
    - *Note, to refine beta-phase, untick ODF refineable on alpha-phase.*
    
- Save this as a template .par file with the experiment number i.e. 'template_065.par' for Diamond 2017 or 'template_103840.par' for Diamond 2021 experiments.

## Structure of notebook and file input/output for Diamond 2017 experiment

**1. Write out a single .esg calibration file for MAUD.**
   - *input: caked synchrotron data for calibration.*
        - data/diamond_2017/calibration-pyFAI/calibration_1200mm_pyFAI.dat
   - *output: calibration .esg data file written to 'calibration' folder, named with detector distance.*
        - analysis/diamond_2017/calibration-pyFAI/calibration_1200mm_pyFAI.esg
        
        
**2. Loop through multiple input files to output .esg data files for MAUD.**
   - *input: caked synchrotron data for experiment.* 
        - data/diamond_2017/065/pixium_03100.dat
   - *output: experiment .esg data file written to 'calibration' folder.*
        - analysis/diamond_2017/065_15deg/065_03100.esg
        
        
**3. Output many .par analysis files for MAUD batch mode analysis.**
   - *input: caked synchrotron data for experiment.* 
        - data/diamond_2017/065/pixium_03100.dat
   - *input: template MAUD .par analysis file.* 
        - analysis/diamond_2017/065_15deg/template_065.par
   - *output: experiment .par analysis files written to 'analysis' folder.*
        - analysis/diamond_2017/065_15deg/065_03100.par
        
        
**4. Write an instruction file for the batch mode.**
   - *output: instruction file.*
        - MAUD_batch_MacOS_065_alpha_20iter.ins
        
        
**5. Running MAUD batch mode.**
   - *input: instruction file.*
        - MAUD_batch_MacOS_065_alpha_20iter.ins
   - *input: experiment .par analysis files.*
        - analysis/diamond_2017/065_15deg/065_03100.par
   - *output: .par analysis files containing refined alpha texture for each experiment.*
        - analysis/diamond_2017/065_15deg/065_03100_alpha_20iter.par   
   - *output: text file containing MAUD refinement results.*
        - analysis/diamond_2017/065_15deg/batch_results_065_alpha_20iter.txt
        
        
**6. Output many .par analysis files to refine the $\beta$ texture.**
   - *input: .par analysis files containing refined alpha texture for each experiment.*
        - analysis/diamond_2017/065_15deg/065_03100_alpha_20iter.par   
   - *output: .par analysis files for refining beta texture.*
        - analysis/diamond_2017/065_15deg/065_03100_beta.par   
        
   
**7. Write an instruction file for refining the $\beta$ texture in batch mode.**
   - *output: instruction file.*
        - MAUD_batch_MacOS_065_beta_20iter.ins
        

**8. Running MAUD batch mode (to refine the $\beta$ texture).**
   - *input: instruction file.*
        - MAUD_batch_MacOS_065_alpha_20iter.ins
   - *input: experiment .par analysis files.*
        - analysis/diamond_2017/065_15deg/065_03100_beta.par 
   - *output: .par analysis files containing refined beta texture for each experiment.*
        - analysis/diamond_2017/065_15deg/065_03100_beta_20iter.par   
   - *output: text file containing MAUD refinement results.*
        - analysis/diamond_2017/065_15deg/batch_results_065_beta_20iter.txt

## Structure of notebook and file input/output for Diamond 2021 experiment

**1. Write out a single .esg calibration file for MAUD.**
   - *input: caked synchrotron data for calibration.*
        - data/diamond_2021/103852-calibration/00001.dat
   - *output: calibration .esg data file written to 'calibration' folder, named with detector distance.*
        - analysis/diamond_2021/103852-calibration/MAUD_calibration_754mm_pyFAI.esg
        
        
**2. Loop through multiple input files to output .esg data files for MAUD.**
   - *input: caked synchrotron data for experiment.* 
        - data/diamond_2021/103845-stage-scan/00001.dat
   - *output: experiment .esg data file written to 'calibration' folder.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001.esg
        
        
**3. Output many .par analysis files for MAUD batch mode analysis.**
   - *input: caked synchrotron data for experiment.* 
        - data/diamond_2021/103845-stage-scan/00001.dat
   - *input: template MAUD .par analysis file.* 
        - analysis/diamond_2021/103845-stage-scan/15deg/template_103845.par
   - *output: experiment .par analysis files written to 'analysis' folder.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001.par
        
        
**4. Write an instruction file for the batch mode.**
   - *output: instruction file.*
        - MAUD_batch_MacOS_103845_alpha_5iter.ins
        
        
**5. Running MAUD batch mode.**
   - *input: instruction file.*
        - MAUD_batch_MacOS_103845_alpha_5iter.ins
   - *input: experiment .par analysis files.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001.par
   - *output: .par analysis files containing refined alpha texture for each experiment.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001_alpha_5iter.par   
   - *output: text file  containing MAUD refinement results.*
        - analysis/diamond_2021/103845-stage-scan/15deg/results/batch_results_103845_alpha_5iter.txt
        
        
**6. Output many .par analysis files to refine the $\beta$ texture.**
   - *input: .par analysis files containing refined alpha texture for each experiment.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001_alpha_5iter.par   
   - *output: .par analysis files for refining beta texture.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001_beta.par   
        
   
**7. Write an instruction file for refining the $\beta$ texture in batch mode.**
   - *output: instruction file.*
        - MAUD_batch_MacOS_103845_beta_5iter.ins
        

**8. Running MAUD batch mode (to refine the $\beta$ texture).**
   - *input: instruction file.*
        - MAUD_batch_MacOS_103845_alpha_5iter.ins
   - *input: experiment .par analysis files.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001_beta.par 
   - *output: .par analysis files containing refined beta texture for each experiment.*
        - analysis/diamond_2021/103845-stage-scan/15deg/103845_00001_beta_5iter.par    
   - *output: text file  containing MAUD refinement results.*
        - analysis/diamond_2021/103845-stage-scan/15deg/results/batch_results_103845_beta_5iter.txt

## Load YAML file

The file paths and user inputs for the analysis of the Diamond 2017 and Diamond 2021 experiments are included in the `yaml` configuration files, to record the inputs of the analysis.

In [ ]:
def get_config(path: str) -> dict:
    """Open a yaml file and return the contents."""
    with open(path) as input_file:
        return yaml.safe_load(input_file)

The information in the `yaml` file can be accessed like this.

In [ ]:
# to load the Diamond 2017 analysis
config_path = "yaml/config_diamond_2017.yaml"
config = get_config(config_path)

In [ ]:
# to load the Diamond 2021 analysis
config_path = "yaml/config_diamond_2021.yaml"
config = get_config(config_path)

In [ ]:
# You can access nested values using multiple sets of square brackets.
print(config["image_numbers"]["start"])
print(config["file_paths"]["input"])

for i in range(3):
    # You can use format to substitute variables into strings
    print(config["file_paths"]["input"].format(test_number=config["user_inputs"]["test_number"], image_number=i))

These simple functions return the file paths as strings.

In [ ]:
def get_io_path(path_stub: str, test_number: int, image_number: int) -> str:
    """Construct the path of the input/output file."""
    return path_stub.format(test_number=test_number, image_number=image_number)

def get_template_path(path_stub: str, test_number: int) -> str:
    """Construct the path of the template file."""
    return path_stub.format(test_number=test_number)
    
def get_image_numbers(start: int, end: int, step: int) -> List[int]:
    return list(range(start, end + 1, step))

In [ ]:
def get_paths(config_path: str, test_number: int, image_number: int) -> Tuple[str, str, str]:
    """Return the path of the input, output and template file."""
    config = get_config(config_path)
    paths = config["file_paths"]
    input_path = get_io_path(paths["input"], test_number, image_number)
    output_esg_path = get_io_path(paths["output_esg"], test_number, image_number)
    output_par_path = get_io_path(paths["output_par"], test_number, image_number)
    template_par_path = get_template_path(paths["template"], test_number)
    
    return(input_path, output_esg_path, output_par_path, template_par_path)

<a id="cell1"></a>
## 1. Write out a single .esg calibration file for MAUD

#### Sample orientation

The goniometer sample rotation angles ${\omega, \chi, \phi}$ refer to sample rotations around the X (RD - rolling direction), Z (ND - normal direction) and Y (TD - transverse direction) axes, respectively. For synchrotron x-ray diffraction, the 0, 0, 0 orientation is such that X (RD) is aligned parrallel with the top of the detector, Z (ND) is aligned along the in-beam direction, Y (TD) is aligned with  the side of the detector. By applying sequential rotations it is possible to align a sample with any given orientation.

In MAUD, the pole figure directions are not given. The pole figure (PF) convention in MAUD is that the right of the PF is the X (RD) direction, or the top of the detector, the top of the PF is the Z (ND) direction, or the in-beam direction, and the centre of the PF is the Y (TD) direction, or the side of the detector. Therefore, it is possible to correctly reorientate the subset pole figure coverage recorded by the diffraction pattern ring intensities during an experiment.

In [ ]:
def dat_to_esg(input_path: str, pixel_size: float, detector_distance: float, start_angle: int, 
               omega: int, chi: int, phi: int, output_path: str):
    """This function reads in a calibration data in '.dat' format and outputs it in '.esg' format
    for reading into MAUD.
    """
    # caked synchrotron data from DAWN
    input_data = np.loadtxt(input_path)

    # read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size

    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes

    # name and open the output MAUD data file
    output_folder = pathlib.Path(output_path).parent
    output_folder.mkdir(exist_ok=True)

    with open(output_path, 'w') as output_file:

        # write metadata for the top of the file
        output_file.write('_pd_block_id noTitle|#0\n'
        '_diffrn_detector Image Plate\n'
        '_diffrn_detector_type Image Plate\n'
        '_pd_meas_step_count_time ?\n'
        '_diffrn_measurement_method ?\n'
        '_diffrn_measurement_distance_unit mm\n'
        '_pd_instr_dist_spec/detc {:.4f}\n'.format(detector_distance))
        output_file.write('_diffrn_radiation_wavelength ?\n'
        '_diffrn_source_target ?\n'
        '_diffrn_source_power ?\n'
        '_diffrn_source_current ?\n')
        output_file.write('_pd_meas_angle_omega {:.1f}\n'.format(omega))# rotation of sample around X (RD) axis
        output_file.write('_pd_meas_angle_chi {:.1f}\n'.format(chi)) # rotation of sample around Z (ND) axis
        output_file.write('_pd_meas_angle_phi {:.1f}\n'.format(phi)) # rotation of sample around Y (TD) axis
        output_file.write('_riet_par_spec_displac_x 0\n'
        '_riet_par_spec_displac_y 0\n'
        '_riet_par_spec_displac_z 0\n'
        '_riet_meas_datafile_calibrated false\n'
        '_pd_meas_angle_eta {:.1f}\n\n'.format(start_angle))
        output_file.write('loop_\n'
        '_pd_proc_2theta_corrected\n'
        '_pd_meas_intensity_total\n')

        # write the first cake intensity data
        for i in range(len(pixel_list) - 1, -1, -1):
            output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][0]))

        # write all the other cake data with additional info
        for cake_number in range(1, number_of_cakes, 1):    
            output_file.write('\n_pd_block_id noTitle|#{:.0f}\n\n'.format(cake_number))
            cake_angle = start_angle + cake_number * cake_width
            output_file.write('_pd_meas_angle_eta {:.1f}\n\n'.format(cake_angle))
            output_file.write('loop_\n'
            '_pd_proc_2theta_corrected\n'
            '_pd_meas_intensity_total\n')

            # write cake intensity data for each cake
            for i in range(len(pixel_list)-1, -1, -1):
                output_file.write('{:.3f}\t{:.8f}\n'.format(pixel_list[i], cake_spectrum[i][cake_number]))

    print(f"Written .esg data file to: '{output_path}'.")

In [ ]:
# user inputs (in mm)
detector_distance_approx = config["user_inputs"]["detector_distance_approx"] # This is an approximate distance which will be refined by MAUD
print("The approximate sample-to-detector distance is: ", detector_distance_approx, " mm")
pixel_size = config["user_inputs"]["pixel_size"]
print("The pixel size of the detector is: ", pixel_size, " mm")

# specifying the angle of the first cake in the data file.
start_angle = config["user_inputs"]["start_angle"]
print("The angle of the first cake in the data file is: ", start_angle, " degrees")

# where the .esg file is written
input_path = config["file_paths"]["input_calibration"]
print("The input path is :", input_path)
output_path = config["file_paths"]["output_calibration"]
print("The output path is :", output_path)

omega = config["user_inputs"]["omega"]
chi = config["user_inputs"]["chi"]
phi = config["user_inputs"]["phi"]

dat_to_esg(input_path, pixel_size, detector_distance_approx, start_angle, omega, chi, phi, output_path)

<a id="cell2"></a>
## 2. Loop through multiple input files to output .esg data files for MAUD

Normally, we have many analysis files spaced in time. These are read in a loop by the `create_data_esgs` which produces one .esg data file for each input .dat file.

In [ ]:
def create_data_esgs(config_path: str, image_numbers: List[int], test_number: int, pixel_size: float, 
                     detector_distance: float, start_angle: int, omega: int, chi: int, phi: int):
    """This function iterates through a set of data files in '.dat' format and outputs them in '.esg' format
    for reading into MAUD.
    """
    # loop through the data using the image number of the files
    for image_number in tqdm(image_numbers):

        # caked synchrotron data from DAWN
        input_path, output_esg_path, output_par_path, template_par_path = get_paths(config_path, test_number, image_number)
        dat_to_esg(input_path, pixel_size, detector_distance, start_angle, omega, chi, phi, output_esg_path)

    print(f"Written {int((end + 1 - start) / step)} .esg data files to '{output_esg_path}' folder.")

In this case the output files are written into the `analysis` folder and saved in the format `{test_number
}_{image_number}.esg`

In [ ]:
detector_distance = config["user_inputs"]["detector_distance"] # use the MAUD calibrated detector distance
print("The exact sample-to-detector distance is: ", detector_distance, " mm")

test_number = config["user_inputs"]["test_number"]
print("The test number is: ", test_number)

start = config["image_numbers"]["start"]
end = config["image_numbers"]["end"]
step = config["image_numbers"]["step"]
print("The start is: ", start, ", the end is ", end, ", the step is ", step)

image_numbers = get_image_numbers(start, end, step)

create_data_esgs(config_path, image_numbers, test_number, pixel_size, detector_distance, start_angle, omega, chi, phi)

## 3. Output many .par analysis files for MAUD batch mode analysis

The next cells can be used to write out a number of .par analysis files to be refined using MAUD batch mode. 


These scripts use a template MAUD .par analysis file to swap the data into. This file will have been manually refined in MAUD and setup ready to refine in batch mode. It is assumed that this template MAUD par file is named in the format `template_{experiment_number}.par`.


The output files are written into an 'analysis' folder and saved in the format `{experiment_number}_{test_number}.par`. This analysis folder will then be used in the subsequent batch mode analysis using MAUD.

In [ ]:
def dat_to_par(input_path: str, pixel_size: float, detector_distance: float, start_angle: int, output_par_path: str,
              template_par_path: str, previous_esg_filename: str, test_number:int, image_number: int):
    """This function reads in data files in '.dat' format and outputs them in '.par' analysis file format
    for analysis using MAUD. The function generates the .par file based on a template.
    """
    # caked synchrotron data from DAWN
    input_data = np.loadtxt(input_path)
    
    # read the pixel spacing from the data file and multiply by pixel size to get pixel positions in mm
    pixel_list = input_data[:, 0] * pixel_size

    cake_spectrum = input_data[:, 1:]
    number_of_cakes = cake_spectrum.shape[1]
    cake_width = 360 / number_of_cakes

    # name and open the output MAUD par analysis file
    output_folder = pathlib.Path(output_par_path).parent
    output_folder.mkdir(exist_ok=True)

    with open(template_par_path, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
        cake_number = 0
        line = template_par_file.readline() 

        while line:
            # rename the datafiles in the .par file
            if previous_esg_filename in line:
                new_esg_filename = f'{test_number}_{image_number}'
                new_line = line.replace(previous_esg_filename, new_esg_filename)
                new_par_file.write(new_line)

            # replace the data when this line is found
            elif '_pd_meas_position _pd_meas_intensity_total _pd_meas_intensity_sigma' in line:
                new_par_file.write(line)

                # write the new data in place
                for j in range(len(pixel_list)-1, -1, -1):
                    template_par_file.readline()
                    new_par_file.write('{:.3f}\t{:.8f}\t1.0\n'.format(pixel_list[j], cake_spectrum[j][cake_number]))
                cake_number += 1

            # write the rest of the file
            else:
                new_par_file.write(line)

            line = template_par_file.readline()
            
    print(f"Written .par analysis file to: '{output_par_path}'.")

In [ ]:
def create_data_pars(config_path: str, image_numbers: List[int], test_number: int, pixel_size: float, 
                     detector_distance: float, start_angle: int, previous_esg_filename: str):
    """This function iterates through a set of data files in '.dat' format and outputs them in '.par' analysis 
    file format for analysis using MAUD.
    """
    # loop through the data using the image number of the files
    for image_number in tqdm(image_numbers):

        # caked synchrotron data from DAWN and MAUD template .par file to swap the data into
        input_path, output_esg_path, output_par_path, template_par_path = get_paths(config_path, test_number, image_number)
        
        dat_to_par(input_path, pixel_size, detector_distance, start_angle, output_par_path, template_par_path,
                  previous_esg_filename, test_number, image_number)

    print(f"Written {int((end + 1 - start) / step)} .par analysis files to: '{output_par_path}'.")

In [ ]:
# name of the data (.esg files) to be replaced in the MAUD template_******.par file
previous_esg_filename = config["user_inputs"]["previous_esg_filename"].format(test_number=config["user_inputs"]["test_number"])

print("To find the data to swop in the template file the script will search for the filename:", previous_esg_filename)

# Do the .par file conversion
create_data_pars(config_path, image_numbers, test_number, pixel_size, detector_distance, start_angle, previous_esg_filename)

## 4.1 Write an instruction file for the batch mode for Mac OS

To run the analysis in batch mode, an instruction file is required to tell MAUD exactly what analysis to do. This section writes the .ins file.

The number 13 corresponds to the refinement analysis in MAUD (texture mode). The number of iterations sets how many times to run the refinement on each sample.

**In Mac OS, we require relative file paths written into the instruction .ins file.**

*Note, in multi-phase spectra only one phase can be refined at a time. In dual-phase Ti alloys, the alpha-phase is refined first, followed by the beta. *

The script to write the instruction file is different for the Diamond 2017 and Diamond 2021 analysis, due to the different file paths contained within the instruction file.

In [ ]:
def create_ins_file_mac_2017(output_ins: str, image_numbers: List[int], test_number: int, 
                             phase: str, number_of_iterations: int):
    """This function creates a '.ins' instruction file for running MAUD in batch mode.
    """
    # name of instruction file
    output_batch_path = pathlib.Path(output_ins)
    
    with output_batch_path.open(mode='w') as output_batch_file:

        output_batch_file.write('loop_\n'
        '    _riet_analysis_file\n'
        '    _riet_analysis_iteration_number\n'
        '    _riet_analysis_wizard_index\n'
        '    _riet_analysis_fileToSave\n'
        '    _riet_append_simple_result_to\n\n')
        # note, _riet_meas_datafile_name\n removed from above for swapping the data files.
        # note, _riet_append_result_to\n removed as this only prints out Rwp value.

        if phase == 'alpha':
            for image_number in image_numbers:
                output_batch_file.write(f"'/analysis/diamond_2017/{test_number:03d}_15deg/{test_number:03d}_{image_number:05d}.par'"
                f" {number_of_iterations} 13 "
                f"'/analysis/diamond_2017/{test_number:03d}_15deg/{test_number:03d}_{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
                f"'/analysis/diamond_2017/{test_number:03d}_15deg/results/batch_results_{test_number:03d}_{phase}_{number_of_iterations}iter.txt\'\n")

        elif phase == 'beta':        
            for image_number in image_numbers:
                output_batch_file.write(f"'/analysis/diamond_2017/{test_number:03d}_15deg/{test_number:03d}_{image_number:05d}_{phase}.par'"
                f" {number_of_iterations} 13 "
                f"'/analysis/diamond_2017/{test_number:03d}_15deg/{test_number:03d}_{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
                f"'/analysis/diamond_2017/{test_number:03d}_15deg/results/batch_results_{test_number:03d}_{phase}_{number_of_iterations}iter.txt\'\n")

    print(".ins file written.")

In [ ]:
test_number = config["user_inputs"]["test_number"]
print("The test number is: ", test_number)

number_of_iterations = config["user_inputs"]["number_of_iterations"]
print("The number of refinement iterations is: ", number_of_iterations)

# which phase is being refined?
phase = config["user_inputs"]["phase1"]
print("The phase is: ", phase)

# number and spacing of files
image_numbers = get_image_numbers(start, end, step)

output_ins = config["file_paths"]["output_mac_ins"].format(test_number=test_number, phase=phase, number_of_iterations=number_of_iterations)
print("The output path of the instruction file is: ", output_ins)

create_ins_file_mac_2017(output_ins, image_numbers, test_number, phase, number_of_iterations)

For the Diamond 2021 analysis, the list of image numbers is slightly different because some of the images needed to be excluded from the batch mode analysis. This is because the diffraction images were recorded after the beam had moved past the edge of the sample.

In [ ]:
def create_ins_file_mac_2021(output_ins: str, image_numbers: List[int], test_number: int, 
                             phase: str, number_of_iterations: int):
    """This function creates a '.ins' instruction file for running MAUD in batch mode.
    """
    # name of instruction file
    output_batch_path = pathlib.Path(output_ins)
    
    with output_batch_path.open(mode='w') as output_batch_file:

        output_batch_file.write('loop_\n'
        '    _riet_analysis_file\n'
        '    _riet_analysis_iteration_number\n'
        '    _riet_analysis_wizard_index\n'
        '    _riet_analysis_fileToSave\n'
        '    _riet_append_simple_result_to\n\n')
        # note, _riet_meas_datafile_name\n removed from above for swapping the data files.
        # note, _riet_append_result_to\n removed as this only prints out Rwp value.

        if phase == 'alpha':
            for image_number in image_numbers:
                output_batch_file.write(f"'/analysis/diamond_2021/{test_number:06d}-stage-scan/15deg/{test_number:06d}_{image_number:05d}.par'"
                f" {number_of_iterations} 13 "
                f"'/analysis/diamond_2021/{test_number:06d}-stage-scan/15deg/{test_number:06d}_{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
                f"'/analysis/diamond_2021/{test_number:06d}-stage-scan/15deg/results/batch_results_{test_number:06d}_{phase}_{number_of_iterations}iter.txt\'\n")

        elif phase == 'beta':        
            for image_number in image_numbers:
                output_batch_file.write(f"'/analysis/diamond_2021/{test_number:06d}-stage-scan/15deg/{test_number:06d}_{image_number:05d}_{phase}.par'"
                f" {number_of_iterations} 13 "
                f"'/analysis/diamond_2021/{test_number:06d}-stage-scan/15deg/{test_number:06d}_{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
                f"'/analysis/diamond_2021/{test_number:06d}-stage-scan/15deg/results/batch_results_{test_number:06d}_{phase}_{number_of_iterations}iter.txt\'\n")

    print(".ins file written.")

In [ ]:
test_number = config["user_inputs"]["test_number"]
print("The test number is: ", test_number)

number_of_iterations = config["user_inputs"]["number_of_iterations"]
print("The number of refinement iterations is: ", number_of_iterations)

# which phase is being refined?
phase = config["user_inputs"]["phase1"]
print("The phase is: ", phase)

# number and spacing of files
image_numbers = np.r_[2:42+1, 45:85+1, 88:128+1, 131:171+1, 174:214+1, 217:257+1, 260:300+1, 303:343+1, 346:386+1]

output_ins = config["file_paths"]["output_mac_ins"].format(test_number=test_number, phase=phase, number_of_iterations=number_of_iterations)
print("The output path of the instruction file is: ", output_ins)

create_ins_file_mac_2021(output_ins, image_numbers, test_number, phase, number_of_iterations)

## 4.2 Write an instruction file for the batch mode for Windows OS [TODO]

To run the analysis in batch mode, an instruction file is required to tell MAUD exactly what analysis to do. This section writes the .ins file.

The number 13 corresponds to the refinement analysis in MAUD (texture mode). The number of iterations sets how many times to run the refinement on each sample.

**In Windows OS, we require an absolute file path for the analysis file in the instruction .ins file.**

*Note, in multi-phase spectra only one phase can be refined at a time. In dual-phase Ti alloys, the alpha-phase is refined first, followed by the beta. *

In [ ]:
def create_ins_file_windows(start: int, end: int, step: int, test_number: int, phase: str, number_of_iterations: int, 
                            file_path: str):
    """This function creates a '.ins' instruction file for running MAUD in batch mode.
    """
    # name of instruction file
    output_batch_path = pathlib.Path(f"../MAUD_batch_WinOS_{test_number:03d}_{phase}_{number_of_iterations}iter.ins")

    with output_batch_path.open(mode='w') as output_batch_file:

        output_batch_file.write('loop_\n'
        '    _riet_analysis_file\n'
        '    _riet_analysis_iteration_number\n'
        '    _riet_analysis_wizard_index\n'
        '    _riet_analysis_fileToSave\n'
        '    _riet_append_simple_result_to\n\n')
        # note, _riet_meas_datafile_name\n removed from above for swapping the data files.
        # note, _riet_append_result_to\n removed as this only prints out Rwp value.

        if phase == 'alpha':
            for image_number in range (start, end + 1, step):
                output_batch_file.write(f"'{file_path}/analysis/{test_number:03d}/MAUD_{test_number:03d}_{image_number:05d}.par' "
                f"{number_of_iterations} 13 'MAUD_{test_number:03d}_{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
                f"'batch_results_{test_number:03d}_{phase}_{number_of_iterations}iter.txt\'\n")

        elif phase == 'beta':        
            for image_number in range (start, end + 1, step):
                output_batch_file.write(f"'{file_path}/analysis/{test_number:03d}/MAUD_{test_number:03d}_{image_number:05d}_{phase}.par' "
                f"{number_of_iterations} 13 'MAUD_{test_number:03d}_{image_number:05d}_{phase}_{number_of_iterations}iter.par' "
                f"'batch_results_{test_number:03d}_{phase}_{number_of_iterations}iter.txt\'\n")

    print(".ins file written.")

In [ ]:
# test number
test_number = 65

# number of refinement iterations
number_of_iterations = 20

# which phase is being refined?
phase = 'alpha'

# number and spacing of files
start = 0
end = 10

step = 1

# what is the file path to the MAUD-batch-analysis folder?
file_path = 'D:/Chris Daniel/MAUD-batch-analysis' # forward or back slashes are fine for the file path as MAUD reads in both

create_ins_file_windows(start, end, step, test_number, phase, number_of_iterations, file_path)

## 5.1 Running MAUD batch mode on Mac OS

Leave the instruction .ins file and a folder for the analysis (containing the newly created .par analysis files) in the same location as it was created from in this notebook, such as; 


**/Users/mbcx9cd4/Documents/GitHub/MAUD-batch-analysis/**


_Note, be careful with using a dropbox folder such as /Users/mbcx9cd4/Dropbox\ \(Research\ Group\)/Lightform\ Postdoc/MAUD_


In the terminal navigate to the hidden files within the maud.app using the command; 


**cd /Applications/Maud.app/Contents/Resources/Java**


_Note, you can view these files by right clicking on show package contents._


Then, run the command to start the batch mode;


**java -mx2048M -cp Maud.jar:miscLib.jar:jgaec.jar:ij.jar com.radiographema.MaudText -f /Users/mbcx9cd4/Documents/GitHub/MAUD-batch-analysis/MAUD_batch_MacOS_103845_alpha_5iter.ins**


- -mx2048M sets the memory to be used i.e. 512, 1024, 2048, etc.
- Maud.jar:miscLib.jar:jgaec.jar:ij.jar is a class path to the jar files.
- An absolute path to the instruction file must be given. MAUD reads in both forward or back slashes for the file path.


The analysis should run automatically. However, sometimes, an *error* appears and the batch mode is cancelled. If this happens, run a command which looks for a file that doesn't exist. A pop-up window will then appear, navigate to the .ins instruction file and click open. 


The batch mode will then run, showing the refinements in the terminal and outputting the refined analysis files and a result text file.

*Note, this was tested with MAUD Version 2.78 on Mac OS Catalina Version 10.15.5* 

## 5.2 Running MAUD batch mode on Windows OS [TODO]

Leave the instruction .ins file and a folder for the analysis (containing the newly created .par analysis files) in the smae location it was created from in this notebook, such as; 


**D:/Chris Daniel/MAUD-batch-analysis/**


_Note, be careful with using a dropbox folder such as /Users/mbcx9cd4/Dropbox\ \(Research\ Group\)/Lightform\ Postdoc/MAUD_


Navigate to the MAUD package, for example; 


**D:/Chris Daniel/Maud**


And place the following `maud_batch.bat` file here.


This contains the command to start the batch mode;


**jdk\bin\java -mx8192M -classpath lib/Maud.jar;lib/ij.jar;lib/jgap.jar;lib/Help.jar;lib/EsquiClient.jar;lib/com.github.tschoonj.xraylib.jar;lib/joone-engine.jar;lib/newt.all.jar;lib/jdic.jar;lib/jdom.jar;lib/sqlite-jdbc.jar;lib/jmol.jar;lib/jgaec.jar;lib$ar;lib/Files.jar;lib/xgridlib.jar;lib/xgridagent.jar;lib/jogl.all.jar;lib/Examples.jar;lib/commons-math.jar;lib/rome.jar;lib/nativewindow.all.jar;lib/Images.jar;lib/swingx.jar;lib/jdic_stub.jar;lib/MySQL-ConnectorJ.jar;lib/HTTPClient.jar;lib/miscLib.jar;lib/gluegen-rt.jar com.radiographema.MaudText -file "D:\Chris Daniel\MAUD-batch-analysis\MAUD_batch_WinOS_065_alpha_20iter.ins"**


- -mx2048M sets the memory to be used i.e. 512, 1024, 2048, etc.
- lib/Maud.jar;lib/ij.jar... is a class path to the jar files.
- An absolute path to the instruction file must be given. MAUD reads in both forward or back slashes for the file path.


The analysis should run automatically. However, sometimes, an *error* appears and the batch mode is cancelled. If this happens on Windows OS, it is not possible to run a command which looks for a file that doesn't exist, since the pop-up window does *not* appear as it does on Mac OS (I'm not sure why). 


The batch mode will then run, showing the refinements in the terminal and outputting the refined analysis files and a result text file.


*Note, this was tested with MAUD Version 2.94 on Windows 7* 

In [ ]:
def create_maud_batch(test_number: int, phase: str, number_of_iterations: int, file_path: str):
    """This function creates a 'maud_batch.bat' batch file to start MAUD in batch mode.
    """
    # name of instruction file
    output_batch_path = pathlib.Path(f"../analysis/batch_files/maud_batch_{test_number:03d}_{phase}_{number_of_iterations}iter.bat")

    with output_batch_path.open(mode='w') as output_batch_file:

        output_batch_file.write(f"jdk\\bin\\java -mx8192M -classpath lib/Maud.jar;lib/ij.jar;lib/jgap.jar;lib/Help.jar;lib/EsquiClient.jar;lib/com.github.tschoonj.xraylib.jar;lib/joone-engine.jar;lib/newt.all.jar;lib/jdic.jar;lib/jdom.jar;lib/sqlite-jdbc.jar;lib/jmol.jar;lib/jgaec.jar;lib$ar;lib/Files.jar;lib/xgridlib.jar;lib/xgridagent.jar;lib/jogl.all.jar;lib/Examples.jar;lib/commons-math.jar;lib/rome.jar;lib/nativewindow.all.jar;lib/Images.jar;lib/swingx.jar;lib/jdic_stub.jar;lib/MySQL-ConnectorJ.jar;lib/HTTPClient.jar;lib/miscLib.jar;lib/gluegen-rt.jar com.radiographema.MaudText -file "
                                f"\"{file_path}/maud_batch_WinOS_{test_number:03d}_{phase}_{number_of_iterations}iter.ins\"")

    print(".bat file written to 'analysis/batch_files/' folder.")

In [ ]:
# test number
test_number = 65

# number of refinement iterations
number_of_iterations = 10

# which phase is being refined?
phase = 'alpha'

# what is the file path to the MAUD-batch-analysis folder?
file_path = 'D:/Chris Daniel/MAUD-batch-analysis' # forward or back slashes are fine for the file path as MAUD reads in both

create_maud_batch(test_number, phase, number_of_iterations, file_path)

## 6. Output many .par analysis files to refine the $\beta$ texture

After refining the alpha texture it is then possible to automatically refine the beta texture using MAUD batch mode. To do this we need to turn off the ODF refinable option for the alpha phase and turn on the ODF refinable option for the beta phase.

In [ ]:
def create_second_phase_pars(input_format_string: str, output_format_string: str, image_numbers: List[int], 
                             test_number: int, phase1: str, phase2: str, number_of_iterations: int):
    """This function reads in an analysis file in '.par' format, which has been refined in MAUD to calculate the 
    phase1 (e.g. alpha) texture. The results of the previous refinement of phase1 are deleted and a refinement
    for phase2 (e.g. beta) is turned on. The file is output in '.par' format for refinement using MAUD.
    """
    for image_number in tqdm(image_numbers):

        second_iteration = False

        # MAUD analysis file with refined alpha texture
        input_par_path = input_format_string.format(test_number=test_number, image_number=image_number, phase=phase1, number_of_iterations=number_of_iterations)
        
        # output MAUD analysis file for refining beta texture
        output_par_path = output_format_string.format(test_number=test_number, image_number=image_number, phase=phase2, number_of_iterations=number_of_iterations)

        with open(input_par_path, 'r') as template_par_file, open(output_par_path, 'w') as new_par_file:
            line = template_par_file.readline() 

            while line:
                # remove the max reflection plane indices from the previous refinement
                if '_refln_index_h _refln_index_k _refln_index_l _refln_F_squared_meas _refln_F_squared_calc \
_refln_F_squared_sigma' in line and second_iteration == False:
                    new_par_file.write(line.strip('\n'))
                    for i in range(0,33):
                        line = template_par_file.readline()
                    second_iteration = True

                # remove the max reflection plane indices from the previous refinement
                elif '_refln_index_h _refln_index_k _refln_index_l _refln_F_squared_meas _refln_F_squared_calc \
_refln_F_squared_sigma' in line and second_iteration == True:
                    new_par_file.write(line.strip('\n'))
                    for i in range(0,12):
                        line = template_par_file.readline()
                    second_iteration = False

                # set refinement of alpha ODF to false
                if '_rita_odf_refinable true' in line:
                    new_par_file.write('_rita_odf_refinable false\n')

                elif '#subordinateObject_none tex' in line:
                    for l in range (0, 7):
                        line = template_par_file.readline()
                    # write metadata for the top of the file
                    new_par_file.write('#subordinateObject_E-WIMV\n\n'
                    '_pd_proc_ls_pref_orient_corr \'E-WIMV\'\n\n'
                    '_rita_generate_symmetry none\n'
                    '_rita_wimv_sum_coincidence true\n'
                    '_rita_wimv_iteration_max 10\n'
                    '_rita_wimv_exponent 0.01\n'
                    '_rita_wimv_refl_min_int 0.001\n'
                    '_rita_wimv_odf_resolution 15.0\n'
                    '_rita_wimv_tube_projection true\n'
                    '_rita_wimv_store_ang_conv true\n'
                    '_rita_wimv_odf_coverage_% 0\n'
                    '_rita_odf_sharpness ?\n'
                    '_rita_wimv_phon_use ?\n'
                    '_rita_wimv_tube_weight 0.5\n'
                    '_rita_wimv_normalize_pole_figures false\n'
                    '_rita_wimv_weigths_exponent 0.5\n'
                    '_rita_odf_refinable true\n'
                    '_rita_wimv_refl_min_dspacing 0.0\n\n'
                    '#custom_object_odf\n'
                    'loop_\n'
                    '_rita_wimv_odf_values\n')

                    for i in range(0, 10):
                        for j in range(0, 10):
                            for k in range (0, 37):
                                new_par_file.write('1.0 ')
                            new_par_file.write('\n')  
                        new_par_file.write('\n')
                    new_par_file.write('\n')

                    new_par_file.write('#end_custom_object_odf\n\n\n'
                    '#end_subordinateObject_E-WIMV\n\n\n')

                # write the rest of the file
                else:
                    new_par_file.write(line)

                line = template_par_file.readline()

    print(f"Written {int((end + 1 - start) / step)} .par analysis files to output folder.")

In [ ]:
test_number = config["user_inputs"]["test_number"]
print("The test number is: ", test_number)

number_of_iterations = config["user_inputs"]["number_of_iterations"]
print("The number of refinement iterations is: ", number_of_iterations)

# which phase has previously been refined?
phase1 = config["user_inputs"]["phase1"]
print("The phase 1 is: ", phase1)
# which phase is being refined?
phase2 = config["user_inputs"]["phase2"]
print("The phase 2, which is being refined, is: ", phase2)

if config_path == "yaml/config_diamond_2021.yaml":
    # number and spacing of files
    image_numbers = np.r_[2:42+1, 45:85+1, 88:128+1, 131:171+1, 174:214+1, 217:257+1, 260:300+1, 303:343+1, 346:386+1]
else:    
    # number and spacing of files
    image_numbers = get_image_numbers(start, end, step)

input_format_string = config["file_paths"]["output_par_phase_refined"]
print("The input path of the par file is: ", input_format_string)
output_format_string = config["file_paths"]["output_par_phase"]
print("The output path of the par file is: ", output_format_string)

create_second_phase_pars(input_format_string, output_format_string, image_numbers, test_number, 
                         phase1, phase2, number_of_iterations)

## 7.1 Write an instruction file for refining the $\beta$ texture in batch mode for Mac OS

We then need to write out the instruction file to run through the refinement of the new beta parameter files and to save the results of the batch analysis.

In [ ]:
test_number = config["user_inputs"]["test_number"]
print("The test number is: ", test_number)

number_of_iterations = config["user_inputs"]["number_of_iterations"]
print("The number of refinement iterations is: ", number_of_iterations)

# which phase is being refined?
phase = config["user_inputs"]["phase2"]
print("The phase is: ", phase)

if config_path == "yaml/config_diamond_2021.yaml":
    # number and spacing of files
    image_numbers = np.r_[2:42+1, 45:85+1, 88:128+1, 131:171+1, 174:214+1, 217:257+1, 260:300+1, 303:343+1, 346:386+1]
else:    
    # number and spacing of files
    image_numbers = get_image_numbers(start, end, step)

output_ins = config["file_paths"]["output_mac_ins"].format(test_number=test_number, phase=phase, number_of_iterations=number_of_iterations)
print("The output path of the instruction file is: ", output_ins)

create_ins_file_mac_2017(output_ins, image_numbers, test_number, phase, number_of_iterations)

## 7.2 Write an instruction and batch file for refining the $\beta$ texture in batch mode for Windows OS [TODO]

In [ ]:
# test number
test_number = 65

# number of refinement iterations
number_of_iterations = 20

# which phase is being refined?
phase = 'beta'

# what is the file path to the MAUD-batch-analysis folder?
file_path = 'D:/Chris Daniel/MAUD-batch-analysis' # forward or back slashes are fine for the file path as MAUD reads in both

create_ins_file_windows(test_number, phase, number_of_iterations, file_path)

create_maud_batch(test_number, phase, number_of_iterations, file_path)